# Fitting Hyperbolas based in locations found with YOLO Model

In [6]:
#import libraries
import numpy as np
from ultralytics import YOLO
import os

#Setting Working directory
import sys
from pathlib import Path

sys.path.append(str(Path.cwd().parent))

#Import the config file so that only the Filename needs to be changed in the _read_segy function
from config import *

In [7]:
#Load trained Model
model = YOLO(YOLO_MODEL_DIR)

In [8]:
from Pipeline.Datatoolkit import DatatoolKit

dk = DatatoolKit(TEST_FILE_DIR , "UG3DQUERUNTERZUG.SGY")
file = dk.LoadSGY()
df = dk.create_df(file)

In [9]:
def extract_bboxes(prediction_folder)-> list:
    results = model.predict(source=prediction_folder, save = True, conf=0.1)
    all_boxes = {}

    for result in results:
        img_name = os.path.basename(result.path)
        if result.boxes is not None:
            boxes = result.boxes.xyxy.cpu().numpy()
            all_boxes[img_name] = boxes# Bounding boxes in (x1, y1, x2, y2) format
        else:
            all_boxes[img_name] = np.empty((0,4))       
        
    return all_boxes

In [10]:
boxes = extract_bboxes(TEST_PIC_DIR)


image 1/3 c:\pythonad\PAINDHS25\PAINDGPR\Data\Testdata\pictures\UG3DQUERUNTERZUG.SGY_inline_52.png: 384x640 5 hyperbolas, 87.5ms
image 2/3 c:\pythonad\PAINDHS25\PAINDGPR\Data\Testdata\pictures\UG3DQUERUNTERZUG.SGY_inline_59.png: 384x640 9 hyperbolas, 78.3ms
image 3/3 c:\pythonad\PAINDHS25\PAINDGPR\Data\Testdata\pictures\frame_0.jpg: 448x640 5 hyperbolas, 82.6ms
Speed: 2.6ms preprocess, 82.8ms inference, 2.0ms postprocess per image at shape (1, 3, 448, 640)
Results saved to C:\pythonad\PAINDHS25\PAINDGPR\Notebooks\runs\detect\predict3


In [19]:
print(boxes)

{'UG3DQUERUNTERZUG.SGY_inline_52.png': array([[      1.137,      107.46,      117.34,      161.32],
       [     6.4071,      71.518,      210.26,      127.78],
       [     1.1274,      56.227,       93.03,      110.63],
       [      1.466,      56.229,      140.63,      111.66],
       [     346.46,      7.3693,      393.12,      54.333]], dtype=float32), 'UG3DQUERUNTERZUG.SGY_inline_59.png': array([[    0.16984,      11.065,      32.777,      55.192],
       [     508.71,      5.6568,       557.3,      50.618],
       [     561.52,      8.3862,      609.23,      50.604],
       [     1.4096,      104.31,      148.92,      164.06],
       [     1.8562,      56.717,      162.21,      106.66],
       [     612.34,      9.4982,      664.65,      50.352],
       [     675.03,      5.3894,         711,      53.813],
       [     62.903,      8.4614,      111.32,       56.05],
       [     401.13,      53.629,      650.72,      107.15]], dtype=float32), 'frame_0.jpg': array([[     151.06,

In [14]:
#translate bboxes coordinates to sgy date

def bbox_to_data(df, bbox:dict,sample_interval=None):

    results = {}

    for img_name, box_list in bbox.items():
        if "_inline_" not in img_name:
            continue
        inline_nr = int(img_name.split("_inline_")[1].split(".")[0])
        results[img_name] = []

        for bbox in box_list:
            x1,y1,x2,y2 = map(int, bbox)

            sub = df[(df["inline"] == inline_nr) & (df["crossline"].between(x1,x2))]
            if sub.empty:
                continue

            amp_slice = np.vstack([
                np.array(row["Amplitude"])[y1:y2] for _, row in sub.iterrows()
            ])

            
            if sample_interval is not None:
                time_start_us = y1 * sample_interval
                time_end_us = y2 * sample_interval
            else:
                time_start_us = time_end_us = None

            results[img_name].append({
                "bbox": bbox,
                "inline": inline_nr,
                "crossline_min": int(sub["crossline"].min()),
                "crossline_max": int(sub["crossline"].max()),
                "sample_start": y1,
                "sample_end": y2,
                "time_start_us": time_start_us,
                "time_end_us": time_end_us,
                "amplitudes": amp_slice
            })

    return results


In [15]:
results = bbox_to_data(df, boxes)

In [18]:
from pprint import pprint
pprint(results)

{'UG3DQUERUNTERZUG.SGY_inline_52.png': [{'amplitudes': array([[  2242,   3370,   1344, ...,  -8469, -11652, -13923],
       [  1111,   2350,    363, ..., -10016, -12566, -13128],
       [  -485,    166,  -1642, ...,  -9353, -10357,  -9309],
       ...,
       [ -2121,    894,   4031, ...,   4640,   4189,   2626],
       [ -4121,  -2184,   1792, ...,   6260,   4493,   2228],
       [ -5373,  -4597,   -341, ...,   6923,   4493,   2387]], shape=(117, 54), dtype=int16),
                                         'bbox': array([      1.137,      107.46,      117.34,      161.32], dtype=float32),
                                         'crossline_max': 117,
                                         'crossline_min': 1,
                                         'inline': 52,
                                         'sample_end': 161,
                                         'sample_start': 107,
                                         'time_end_us': None,
                                         